## CCG算法算例复现

原问题描述
$$
\begin{array}{lll}
\min & 400 y_{0}+414 y_{1}+326 y_{2} & \\
& +18 z_{0}+25 z_{1}+20 z_{2} & \\
& +22 x_{00}+33 x_{01}+24 x_{02}+33 x_{10}+23 x_{11} & \\
& +30 x_{12}+20 x_{20}+25 x_{21}+27 x_{22} & \forall i=0,1,2 ; \\
\text { s.t. } & z_{i} \leq 800 y_{i}, & \forall i=0,1,2 ; \\
& \sum_{j} x_{i j} \leqslant z_{i}, & \forall j=0,1,2 ; \\
& \sum_{i} x_{i j} \geqslant d_{j}, & \forall i=0,1,2 ; \\
& y_{i} \in\{0,1\}, z_{i} \geqslant 0, & \forall i=0,1,2 ;, \forall j=0,1,2 ;
\end{array}
$$

不确定集
$$
\begin{aligned}
\mathbf{D}=&\left\{\mathbf{d}: d_{0}=206+40 g_{0}, d_{1}=274+40 g_{1}, d_{2}=220+40 g_{2},\right.\\
& 0 \leq g_{0} \leq 1,0 \leq g_{1} \leq 1,0 \leq g_{2} \leq 1, \\
&\left.g_{0}+g_{1}+g_{2} \leq 1.8, g_{0}+g_{1} \leq 1.2\right\} .
\end{aligned}
$$

首先,我们导入gurobipy库和numpy库

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

随后,我们对常数项进行命名和初始化:

- 原目标函数可以简化为$\min fy + az + Cx$

- 不确定集合可以简写为$d=dl+du\times g$

In [2]:
# Constant Setting
f = [400, 414, 326]
a = [18, 25, 20]
C = [[22, 33, 24],
    [33, 23, 30],
    [20, 25, 27]]
dl = [206, 274, 220]
du = [40, 40, 40]
k = 0 # count iteration

下面将原问题拆解为主问题MP和子问题SP进行求解:

主问题
$$
    \begin{align}
    \min & \sum_i f_i y_i + \sum_i a_i z_i + \eta &\\
    s.t. & z_{i} \leq 800 y_{i}, & \forall i=0,1,2  \\
        & \sum_i z_i \geq 772 \\
        & \sum_{j} x_{i j}^l \leqslant z_{i}, & \forall j=0,1,2 ; \\
        & \sum_{i} x_{i j}^l \geqslant d_{j}^l, & \forall i=0,1,2 ; \\ 
        & \eta \geq \sum_i \sum_j c_{i j}x_{i j}^l, & \forall i,j=0,1,2\quad \forall l=1,2,...;\\
        & y_i \in \{0,1\}, z_i\geq0, x_{i j} \geq 0 & \forall i,j=0,1,2
    \end{align}
$$



In [3]:
MP = gp.Model()
x = MP.addVars(3,3, lb=0, vtype=GRB.CONTINUOUS, name='x_0')
y = MP.addVars(len(f), lb=0, ub=1, obj=f, vtype=GRB.BINARY, name='y')
z = MP.addVars(len(a), lb=0, obj=a, vtype=GRB.CONTINUOUS, name='z')
g = MP.addVars(3, lb=0, ub=1, name='g')
d = MP.addVars(3, lb=0, name='d')
eta = MP.addVar(obj=1, lb=0, name='η')

MP_Cons_1 = MP.addConstrs((z[i] <= 800*y[i] for i in range(3)), name='MP_Cons_1')
MP_Cons_2 = MP.addConstr((gp.quicksum(z[i] for i in range(3)) >= 772), name='MP_Cons_2')

# iteration constraints
MP_Cons_3 = MP.addConstrs((gp.quicksum(x[i,j] for j in range(3)) <= z[i] for i in range(3)), name='MP_Cons_3')
MP_Cons_4 = MP.addConstrs((gp.quicksum(x[i, j] for i in range(3)) >= d[j] for j in range(3)), name='MP_Cons_4')
MP_Cons_eta = MP.addConstr(eta >= gp.quicksum(x[i,j]*C[i][j] for i in range(3) for j in range(3)), name='MP_Cons_eta')


# Master-problem uncertainty constraints
MP_Cons_uncertainty_1 = MP.addConstrs((d[i] == dl[i]+du[i]*g[i] for i in range(3)), name='MP_Uncertainty_Cons1')
MP_Cons_uncertainty_2 = MP.addConstr((gp.quicksum(g[i] for i in range(3)) <= 1.8), name='MP_Uncertainty_Cons2')
MP_Cons_uncertainty_3 = MP.addConstr((gp.quicksum(g[i] for i in range(2)) <= 1.2), name='MP_Uncertainty_Cons3')

MP.optimize()

LB = MP.objval

Academic license - for non-commercial use only - expires 2023-05-14
Using license file C:\Users\liuzi\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 16 rows, 22 columns and 54 nonzeros
Model fingerprint: 0xa1e1e845
Variable types: 19 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 6 rows and 7 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 30 nonzeros
Variable types: 12 continuous, 3 integer (3 binary)

Root relaxation: objective 3.145999e+04, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 31459.9896    0    2          - 31459.9896      -     -    0s
H  


子问题
$$
    \begin{align}
        Q(y) = &\max_{d\in D} \min_x \sum_i \sum_j c_{i j}x_{i j} \\
        s.t. & \sum_j x_{i j} \leq z_i^*\\
            & \sum_i x_{i j} \geq d_j\\
            & x_{i j} \geq 0 
    \end{align}
$$

需要利用KKT条件对该问题的Inner-level进行改写,改为只含约束条件的线性方程求解,再加上外层的优化.

$$
\begin{array}{rlr}
\mathbf{S P}_{2}: \mathcal{Q}(\mathbf{y})= & \max \sum_{i} \sum_{j} c_{i j} x_{i j} & \\
\text { s.t. } & \sum_{j} x_{i j} \leqslant z_{i}^{*}, & \forall i=0,1,2 ; \quad \rightarrow \pi \\
& \sum_{i}-x_{i j} \leqslant-d_{j}, & \forall j=0,1,2 ; \quad \rightarrow \theta \\
& \pi_{i}-\theta_{j} \leqslant c_{i j}, & \forall i=0,1,2 ; \forall j=0,1,2 ; \quad \rightarrow x_{i j} \\
& \left(z_{i}^{*}-\sum_{j} x_{i j}\right) \pi_{i}=0, & \forall i=0,1,2 ; \\
& \left(\sum_{i} x_{i j}-d_{j}\right) \theta_{j}=0, & \forall j=0,1,2 ; \\
& \left(c_{i j}-\pi_{i}+\theta_{j}\right) x_{i j}=0, & \forall i=0,1,2 ; \forall j=0,1,2 \\
&d_{j}=\underline{d}_{j}+g_{j} \tilde{d}_{j} & \forall j=0,1,2 ; \\
&\sum_{j} g_{j} \leqslant \Gamma, & j=1, \cdots, n \\
&g_{j} \in[0,1], & \forall j=1, \cdots, n \\
&\pi_{i} \leqslant 0, & \forall i=0,1,2 ; \\
&\theta_{j} \leqslant 0, & \forall j=0,1,2 ; \\
&d_{j} \geqslant 0, & \forall j=0,1,2 ;
\end{array}
$$

将其中三个非线性约束进行转化,利用大M法即可得到:

$$
\begin{aligned}
&\mathbf{S P}_{2}: \mathcal{Q}(\mathbf{y})=\max \sum_{i} \sum_{j} c_{i j} x_{i j}\\
&\pi_{i}-\theta_{j} \leqslant c_{i j}, \quad \forall i=0,1,2 ; \forall j=0,1,2 ; \quad \rightarrow x_{i j}\\
&-\pi_{i} \leqslant M v_{i}, \quad \forall i=0,1,2 \text {; }\\
&z_{i}^{*}-\sum_{j} x_{i j} \leqslant M\left(1-v_{i}\right) \quad \forall i=0,1,2 \text {; }\\
&-\theta_{j} \leqslant M w_{j}, \quad \forall j=0,1,2 ;\\
&\sum_{i} x_{i j}-d_{j} \leqslant M\left(1-w_{j}\right), \quad \forall j=0,1,2 \text {; }\\
&x_{i j} \leqslant M h_{i j}, \quad \forall i=0,1,2 ; \forall j=0,1,2 \text {; }\\
&c_{i j}-\pi_{i}+\theta_{j} \leqslant M\left(1-h_{i j}\right) \quad \forall i=0,1,2 ; \forall j=0,1,2 \text {; }\\
&v_{i}, w_{j}, h_{i, j} \in\{0,1\} . \quad \forall i=0,1,2 ; \forall j=0,1,2 \text {; }\\
&\begin{array}{ll}
d_{j}=\underline{d}_{j}+g_{j} \tilde{d}_{j} & \forall j=0,1,2 ; \\
\sum_{j} g_{j} \leqslant \Gamma, & j=1, \cdots, n
\end{array}\\
&g_{j} \in[0,1], \quad j=1, \cdots, n\\
&\pi_{i} \leqslant 0, \quad \forall i=0,1,2 ;\\
&\begin{array}{ll}
\theta_{j} \leqslant 0, & \forall j=0,1,2 ; \\
d_{j} \geqslant 0, & \forall j=0,1,2 ;
\end{array}
\end{aligned}
$$

In [4]:
SP = gp.Model()
x_sub = SP.addVars(3, 3, lb=0, vtype=GRB.CONTINUOUS, name='x_sub')
d_sub = SP.addVars(3, lb=0, name='d_sub')
g_sub = SP.addVars(3, lb=0, ub=1, name='g_sub')
pi = SP.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name='pi')
v = SP.addVars(6, vtype=GRB.BINARY, name='v')
w = SP.addVars(3, 3, vtype=GRB.BINARY, name='w')
M = 10000

# Constraints
SP_Cons_1 = SP.addConstrs((gp.quicksum(x_sub[i,j] for j in range(3)) <= z[i].x for i in range(3)), name='SP_Cons_1') #[0:3]
SP_Cons_2 = SP.addConstrs((gp.quicksum(x_sub[i,j] for i in range(3)) >= d_sub[j] for j in range(3)), name='SP_Cons_2') #[3:6]
SP_Cons_3 = SP.addConstrs((-pi[i]+pi[j+3] <= C[i][j] for i in range(3) for j in range(3)), name='SP_Cons_3') #[6:15]

# slack constraints part 1
SP_SLACK_CONS_1 = SP.addConstrs((z[i].x-gp.quicksum(x_sub[i,j] for j in range(3)) <= M*(1-v[i]) for i in range(3)), name='SP_SLACK_CONS_1') #[15:18]
SP_SLACK_CONS_2 = SP.addConstrs((gp.quicksum(x_sub[i,j] for i in range(3))-d_sub[j] <= M*(1-v[j+3]) for j in range(3)), name='SP_SLACK_CONS_2') #[18:21]
SP_SLACK_CONS_3 = SP.addConstrs((pi[i] <= M*v[i] for i in range(6)), name='SP_SLACK_CONS_3') # [21:27]

# slack constraints part 2
SP_SLACK_CONS_4 = SP.addConstrs((C[i][j]+pi[i]-pi[j+3] <= M*(1-w[i,j]) for i in range(3) for j in range(3)), name='SP_SLACK_CONS_4') # [27:36]
SP_SLACK_CONS_5 = SP.addConstrs((x_sub[i,j] <= M*w[i,j] for i in range(3) for j in range(3)), name='SP_SLACK_CONS_5') # [36:45]

# uncertainty
SP_Cons_uncertainty_1 = SP.addConstrs((d_sub[i] == dl[i]+du[i]*g_sub[i] for i in range(3)), name='SP_Uncertainty_Cons1') #[45:48]
SP_Cons_uncertainty_2 = SP.addConstr((gp.quicksum(g_sub[i] for i in range(3)) <= 1.8), name='MP_Uncertainty_Cons2') #[48]
SP_Cons_uncertainty_3 = SP.addConstr((gp.quicksum(g_sub[i] for i in range(2)) <= 1.2), name='MP_Uncertainty_Cons3') #[49]


sub_obj = gp.quicksum(C[i][j]*x_sub[i,j] for i in range(3) for j in range(3))
SP.setObjective(sub_obj, GRB.MAXIMIZE)
SP.optimize()
SP_objval = SP.objval
UB = LB - eta.x + SP_objval
print(UB)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 50 rows, 36 columns and 134 nonzeros
Model fingerprint: 0xe0c5f711
Variable types: 21 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 16662.000000
Presolve removed 24 rows and 16 columns
Presolve time: 0.01s
Presolved: 26 rows, 20 columns, 71 nonzeros
Variable types: 13 continuous, 7 integer (7 binary)

Root relaxation: objective 2.165400e+04, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 21654.0000    0    2 16662.0000 21654.0000  30.0%     -    0s
H    0     0                    18326.000000 21654.0000  18.2

随后,开始CCG算法的迭代,其伪代码为:

Column-and-constraint generation (C\&CG) algorithm
1. Set $L B=-\infty, U B=+\infty, k=0$ and $\mathbf{0}=\emptyset$.
2. Solve the following master problem.
$$
\begin{aligned}
&\mathbf{M P}_{2}: \min _{\mathbf{y}, \eta} \mathbf{c}^{T} \mathbf{y}+\eta \\
&\text { s.t. } \mathbf{A y} \geq \mathbf{d} \\
&\eta \geq \mathbf{b}^{T} \mathbf{x}^{l}, \quad \forall l \in \mathbf{0} \\
&\mathbf{E y}+\mathbf{G} \mathbf{x}^{l} \geq \mathbf{h}-\mathbf{M} u_{l}^{*}, \quad \forall l \leq k \\
&\mathbf{y} \in \mathbf{S}_{\mathbf{y}}, \quad \eta \in \mathbb{R}, \quad \mathbf{x}^{l} \in \mathbf{S}_{\mathbf{x}} \quad \forall l \leq k
\end{aligned}
$$
Derive an optimal solution $\left(\mathbf{y}_{k+1}^{*}, \eta_{k+1}^{*}, \mathbf{x}^{1 *}, \ldots, \mathbf{x}^{k *}\right)$ and update $L B=\mathbf{c}^{T} \mathbf{y}_{k+1}^{*}+\eta_{k+1}^{*}$.

3. Call the oracle to solve subproblem $\mathbf{S P}_{2}$ in $(10)$ and update $U B=$ $\min \left\{U B, \mathbf{c}^{T} \mathbf{y}_{k+1}^{*}+\mathcal{Q}\left(\mathbf{y}_{k+1}^{*}\right)\right\}$
4. If $U B-L B \leq \epsilon$, return $\mathbf{y}_{k+1}^{*}$ and terminate. Otherwise, do
(a) if $\mathcal{Q}\left(\mathbf{y}_{k+1}^{*}\right)<+\infty$, create variables $\mathbf{x}^{k+1}$ and add the following constraints
$\eta \geq \mathbf{b}^{T} \mathbf{x}^{k+1}$
$\mathbf{E y}+\mathbf{G} \boldsymbol{x}^{k+1} \geq \mathbf{h}-\mathbf{M} u_{k+1}^{*}$
to $\mathbf{M P}_{2}$ where $u_{k+1}^{*}$ is the optimal scenario solving $\mathcal{Q}\left(\mathbf{y}_{k+1}^{*}\right)$. Update $k=k+1, \mathbf{O}=\mathbf{O} \cup\{k+1\}$ and go to Step 2 .

In [5]:
while np.abs(UB-LB)>1e-5 :
    k = k+1
    # Master-problem
    x_new = MP.addVars(3,3, lb=0, vtype=GRB.CONTINUOUS, name='x_{0}'.format(k))
    MP_Cons_3_new = MP.addConstrs((gp.quicksum(x_new[i,j] for j in range(3)) <= z[i] for i in range(3)), name='MP_Cons_3_{}'.format(k))
    MP_Cons_4_new = MP.addConstrs((gp.quicksum(x_new[i, j] for i in range(3)) >= d_sub[j].x for j in range(3)), name='MP_Cons_4_{}'.format(k))
    MP_Cons_eta = MP.addConstr(eta >= gp.quicksum(x_new[i,j]*C[i][j] for i in range(3) for j in range(3)), name='MP_Cons_eta_{}'.format(k))
    MP.optimize()
    LB = max(LB, MP.objval)
    print(LB)

    # Sub-problem update
    # delete old constraints which related to z
    SP.remove(SP.getConstrs()[0:3])
    SP.remove(SP.getConstrs()[15:18])
    # add new constraints which related to z
    SP_Cons_1 = SP.addConstrs((gp.quicksum(x_sub[i,j] for j in range(3)) <= z[i].x for i in range(3)), name='SP_Cons_1') #[0:3]
    SP_SLACK_CONS_1 = SP.addConstrs((z[i].x-gp.quicksum(x_sub[i,j] for j in range(3)) <= M*(1-v[i]) for i in range(3)), name='SP_SLACK_CONS_1') #[15:18]
    
    SP.optimize()
    UB = LB - eta.x + SP.objval
    print(UB)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 23 rows, 31 columns and 85 nonzeros
Model fingerprint: 0xb5a4aa5d
Variable types: 28 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]

MIP start from previous solve produced solution with objective 33680 (0.01s)
Loaded MIP start from previous solve with objective 33680

Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolved: 18 rows, 25 columns, 71 nonzeros
Variable types: 22 continuous, 3 integer (3 binary)

Root relaxation: objective 3.329190e+04, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      33680.0000 33680.00

最终输出求解结果:

In [7]:
print("Iteration finished! We found the optimal solution!")
print("Final Objective:{0}".format(LB))
print(y[0],y[1],y[2])
print(z[0],z[1],z[2])
for i in range(3):
    for j in range(3):
        print(x_new[i,j])

Iteration finished! We found the optimal solution!
Final Objective:33680.0
<gurobi.Var y[0] (value 1.0)> <gurobi.Var y[1] (value 0.0)> <gurobi.Var y[2] (value 1.0)>
<gurobi.Var z[0] (value 458.0)> <gurobi.Var z[1] (value 0.0)> <gurobi.Var z[2] (value 314.0)>
<gurobi.Var x_1[0,0] (value 206.0)>
<gurobi.Var x_1[0,1] (value 0.0)>
<gurobi.Var x_1[0,2] (value 252.0)>
<gurobi.Var x_1[1,0] (value 0.0)>
<gurobi.Var x_1[1,1] (value 0.0)>
<gurobi.Var x_1[1,2] (value 0.0)>
<gurobi.Var x_1[2,0] (value 0.0)>
<gurobi.Var x_1[2,1] (value 314.0)>
<gurobi.Var x_1[2,2] (value 0.0)>
